In [325]:
#installing the packages
#!pip install BeautifulSoup4
#!pip install lxml
#!pip install html5lib
#!pip install geocoder
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab

Solving environment: done

## Package Plan ##

  environment location: /opt/ibm/conda/miniconda3

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.0.2p             |       h470a237_0         3.5 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.3.0               |             py_0          24 KB  conda-forge
    certifi-2018.8.24          |           py35_1         138 KB  conda-forge
    ca-certificates-2018.8.24  |       ha4d7672_0         136 KB  conda-forge
    altair-2.2.2               |             py_0         278 KB  conda-forge
    conda-4.5.11               |           py35_0         636 KB  conda-forge
    ------------------------------------------------------------
                      

In [329]:
#loading the liberaries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import geocoder
from sklearn.cluster import KMeans
import folium # map rendering library
import matplotlib.cm as cm
import matplotlib.colors as colors

In [282]:
canadapostalcodes = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [283]:
#loading the request url
canadapostalcodeshtml = requests.get(canadapostalcodes).text
soup = BeautifulSoup(canadapostalcodeshtml,'lxml')

In [284]:
#loading the  table data
matched_data = soup.find_all('table', class_="wikitable sortable")

In [285]:
#defining the table columns
df = pd.DataFrame(columns=[th.text.replace('\n','') for th in matched_data[0].find_all('tr')[0].find_all('th')])

In [286]:
#loading the data for each column
df[df.columns[0]] = [tr.find_all('td')[0].text.replace('\n','') for tr in matched_data[0].find_all('tr') if len(tr.find_all('td'))>0]
df[df.columns[1]] = [tr.find_all('td')[1].text.replace('\n','') for tr in matched_data[0].find_all('tr') if len(tr.find_all('td'))>0]
df[df.columns[2]] = [tr.find_all('td')[2].text.replace('\n','') for tr in matched_data[0].find_all('tr') if len(tr.find_all('td'))>0]

In [287]:
#filtering the Not assigned data and null data
complete_info_df =df[(df[df.columns[0]] != 'Not assigned') & (df[df.columns[1]] != 'Not assigned') & (df[df.columns[2]] != 'Not assigned')]
complete_info_df=complete_info_df.dropna()
complete_info_df.reset_index(drop=True, inplace=True)
#pd.set_option('display.max_rows', 500)

In [290]:
#merging the repeated
complete_info_df=complete_info_df.groupby([complete_info_df.columns[0],complete_info_df.columns[1]], as_index=False)['Neighbourhood'].agg(','.join)

In [291]:
complete_info_df.Neighbourhood = complete_info_df.Neighbourhood.apply(lambda x: x.replace(']',''))
complete_info_df.shape
#completed assignment 1

(102, 3)

In [292]:
#geocoder.google('M1B , Scarborough , Toronto , Ontario, Canada').latlng

In [293]:
#getting the lattiude and longtidude
longlatlist = complete_info_df.apply(lambda x:geocoder.google('{}, {}, Toronto, Ontario'.format(x['Postcode'], x['Borough'])).latlng, axis=1)
#longlatlist

In [294]:
complete_info_df['LongLat'] = longlatlist

In [295]:
complete_info_df.dropna(inplace=True)

In [296]:
complete_info_df['Lontiude'] = complete_info_df.LongLat.apply(lambda x: x[1])

In [297]:
complete_info_df['Latitude'] = complete_info_df.LongLat.apply(lambda x: x[0])

In [298]:
complete_info_df.drop(columns='LongLat',inplace=True)

In [299]:
#complete_info_df

In [300]:
#end of assignment 2

In [301]:
# The code was removed by Watson Studio for sharing.

In [302]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [303]:
# type your answer here
LIMIT=500
Toronto_venues = getNearbyVenues(names=complete_info_df['Neighbourhood'],
                                   latitudes=complete_info_df['Latitude'],
                                   longitudes=complete_info_df['Lontiude']
                                  )
    

Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
CFB Toronto,Downsview East
Downsview Central
Woodbine Heights
Leaside
Thorncliffe Park
East Toronto
The Beaches West,India Bazaar
Studio District
Davisville North
Davisville
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Harbourfront East,Toronto Islands,Union Station
Bedford Park,Lawrence Manor East
Roselawn
Forest Hill North,Forest Hill West


In [304]:
# one hot encoding
Toronto_venues_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_venues_onehot['Neighbourhood'] = Toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_venues_onehot.columns[-1]] + list(Toronto_venues_onehot.columns[:-1])
Toronto_venues_onehot = Toronto_venues_onehot[fixed_columns]

Toronto_venues_onehot.head()

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek,Rouge Hill,Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [305]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [312]:
Toronto_venues_grouped = Toronto_venues_onehot.groupby('Neighbourhood').mean().reset_index()
Toronto_venues_grouped

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
1,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
2,"Alderwood,Long Branch",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
3,"Bathurst Manor,Downsview North,Wilson Heights",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.050000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
4,Bayview Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
5,"Bedford Park,Lawrence Manor East",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
6,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
7,"Birch Cliff,Cliffside West",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
8,"Bloordale Gardens,Eringate,Markland Wood,Old B...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
9,"Brockton,Exhibition Place,Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.018519,0.000000,0.000000,0.037037,0.00,0.000000,0.000000,0.000000,0.018519


In [313]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = Toronto_venues_grouped['Neighbourhood']

for ind in np.arange(Toronto_venues_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_venues_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Lounge,Breakfast Spot,Print Shop,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop
1,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Coffee Shop,Pizza Place,Sandwich Place,Fast Food Restaurant,Fried Chicken Joint,Beer Store,Pharmacy,College Stadium,Department Store
2,"Alderwood,Long Branch",Pizza Place,Gym,Athletics & Sports,Pub,Dance Studio,Sandwich Place,Bank,Coffee Shop,General Travel,Dumpling Restaurant
3,"Bathurst Manor,Downsview North,Wilson Heights",Coffee Shop,Chinese Restaurant,Sushi Restaurant,Pet Store,Pharmacy,Pizza Place,Middle Eastern Restaurant,Deli / Bodega,Bridal Shop,Restaurant
4,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Yoga Studio
5,"Bedford Park,Lawrence Manor East",Italian Restaurant,Coffee Shop,Fast Food Restaurant,Sushi Restaurant,Juice Bar,Thai Restaurant,Restaurant,Indian Restaurant,Pub,Butcher
6,Berczy Park,Coffee Shop,Cocktail Bar,Café,Seafood Restaurant,Bakery,Beer Bar,Restaurant,Cheese Shop,Steakhouse,Farmers Market
7,"Birch Cliff,Cliffside West",Skating Rink,College Stadium,Café,General Entertainment,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
8,"Bloordale Gardens,Eringate,Markland Wood,Old B...",Liquor Store,Pizza Place,Café,Beer Store,Pharmacy,Comedy Club,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
9,"Brockton,Exhibition Place,Parkdale Village",Bar,Grocery Store,Coffee Shop,Electronics Store,Park,Café,Italian Restaurant,Arts & Crafts Store,Vietnamese Restaurant,Furniture / Home Store


In [314]:
num_top_venues = 5

for hood in Toronto_venues_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = Toronto_venues_grouped[Toronto_venues_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
               venue  freq
0             Lounge  0.25
1       Skating Rink  0.25
2     Breakfast Spot  0.25
3         Print Shop  0.25
4  Accessories Store  0.00


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
            venue  freq
0   Grocery Store  0.22
1      Beer Store  0.11
2     Pizza Place  0.11
3     Coffee Shop  0.11
4  Sandwich Place  0.11


----Alderwood,Long Branch----
            venue  freq
0     Pizza Place  0.22
1             Gym  0.11
2  Sandwich Place  0.11
3             Pub  0.11
4            Bank  0.11


----Bathurst Manor,Downsview North,Wilson Heights----
                       venue  freq
0                Coffee Shop  0.10
1  Middle Eastern Restaurant  0.05
2                Supermarket  0.05
3         Frozen Yogurt Shop  0.05
4           Sushi Restaurant  0.05


----Bayview Village----
                 venue  freq
0                 Café  0.25
1   Chinese Restaurant  0.25
2  Japan

In [315]:
# set number of clusters
kclusters = 5

Toronto_venues_grouped_clustering = Toronto_venues_grouped.drop('Neighbourhood', 1)



# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_venues_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [316]:
len(kmeans.labels_)

69

In [318]:
len(neighborhoods_venues_sorted)

69

In [321]:
complete_info_df = complete_info_df[:-3]

In [322]:
toronto_merged = complete_info_df



# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

/opt/ibm/conda/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Postcode,Borough,Neighbourhood,Lontiude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",-79.194353,43.806686,0,Fast Food Restaurant,Yoga Studio,Department Store,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",-79.160497,43.784535,0,Bar,Yoga Studio,Department Store,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",-79.188711,43.763573,0,Electronics Store,Medical Center,Breakfast Spot,BBQ Joint,Mexican Restaurant,Rental Car Location,Empanada Restaurant,Eastern European Restaurant,Dumpling Restaurant,Drugstore
3,M1G,Scarborough,Woburn,-79.216917,43.770992,0,Coffee Shop,Korean Restaurant,Mexican Restaurant,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant
4,M1H,Scarborough,Cedarbrae,-79.239476,43.773136,0,Hakka Restaurant,Caribbean Restaurant,Athletics & Sports,Thai Restaurant,Bank,Bakery,Fried Chicken Joint,Diner,Discount Store,Dog Run


In [331]:
# create map
latitude=43.806686
longitude = -79.194353
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Lontiude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters